In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

import matplotlib.cm as cm
    
import sys 
sys.path.insert(0, '../src/')

from astropy.table import Table
import galhalo
import anaclass
import os
import warnings; warnings.simplefilter('ignore')
from multiprocessing import Pool
import time

import corner
import sampyl as smp

In [7]:
mass_bins = np.linspace(4,11,45)

def SLFmodel(alpha, delta, sigma):

    lgMhs = np.load("../../data/3000_12_8/truth_lgMh.npy")

    lgMs_mat = galhalo.master_SHMR_1D(lgMhs, alpha, delta, sigma) # will be a 3D array if sigma is non zero

    count_mat = np.zeros(shape=(lgMs_mat.shape[1], lgMs_mat.shape[0], mass_bins.shape[0]))
    for i in range(lgMs_mat.shape[1]): # to measure a cumulative mass function for each random sample
        count_mat[i] = np.apply_along_axis(anaclass.cumulative, 1, lgMs_mat[:,i,:], mass_bins=mass_bins)

    counts = np.average(count_mat, axis=0) # to average across all the random samples
    quant = np.percentile(counts, np.array([5, 50, 95]), axis=0, method="closest_observation") # median and scatter

    S1 = quant[2, 18] - quant[0, 18] #18, 22, 26 corresponds to 6.5, 7, 7.5 Msol
    S2 = quant[2, 22] - quant[0, 22]
    S3 = quant[2, 26] - quant[0, 26]
    N1 = quant[1, 18]
    N2 = quant[1, 22]
    N3 = quant[1, 26]

    model = np.array([N1, N2, N3, S1, S2, S3])

    return model

In [8]:
def lnlike(alpha, delta, sigma):
    model = SLFmodel(alpha, delta, sigma)

    X_0 = np.array([6., 4., 2., 8., 4., 3.])
    X_0_err = np.array([0.18850575, 0.09310345, 0.06436782, 0.7862069 , 0.60229885, 0.44712644])

    chisquare = (X_0 - model)**2/X_0_err**2 
    return -1/2*np.sum(chisquare)

In [9]:
def lnprior(alpha, delta, sigma):
    if 1.5 < alpha < 3.0 and -0.5 < delta < 1.5 and 0 < sigma < 2:
        return 0.0
    return -np.inf

In [10]:
def lnprob(alpha, delta, sigma):
    lp = lnprior(alpha, delta, sigma)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(alpha, delta, sigma)

In [11]:

start = {'alpha': 1.8, 'delta': 0.2, 'sigma': 0.5}
metro = smp.Metropolis(lnprob, start)
chain = metro.sample(200, burn=50)

AttributeError: module 'collections' has no attribute 'Hashable'